In [ ]:
from function_library import *
from reformulation_library import *

In [ ]:
different_validation_losses = False
n_holdout = 1000
n_test = 1000
grid_dim = 5
p_features = 5
n_train = 1000
polykernel_degree = 1
polykernel_noise_half_width = 0
d_feasibleregion = 2 * p_features * (p_features - 1)
sources, destinations = convert_grid_to_list(grid_dim, grid_dim)
sp_graph = shortest_path_graph(sources = sources, destinations = destinations,
        start_node = 1, end_node = grid_dim^2, acyclic = True)
B_true = np.array([[bernoulli(0.5) for k in range(p_features)] for e in range(d_feasibleregion)])
X_train, c_train = generate_poly_kernel_data_simple(B_true, n_train, polykernel_degree, polykernel_noise_half_width)
X_validation, c_validation = generate_poly_kernel_data_simple(B_true, n_holdout, polykernel_degree, polykernel_noise_half_width)
X_test, c_test = generate_poly_kernel_data_simple(B_true, n_test, polykernel_degree, polykernel_noise_half_width)

In [13]:
from sklearn.ensemble import RandomForestRegressor


@dataclass
class rf_graph:
    num_trees : int = 100
    num_features_per_split : int = -1

def train_random_forests_po(X: np.ndarray, c: np.ndarray, rf_alg_parms=rf_graph()):
    num_trees = rf_alg_parms.__dict__['num_trees']
    num_features_per_split = rf_alg_parms.__dict__['num_features_per_split']

    p, n = X.shape
    d, n2 = c.shape
    if n != n2:
        raise ValueError("Dimensions of the input are mismatched.")
    X_t = X.T
    # If num_features_per_split is not specified, use default for regression
    if num_features_per_split < 1:
        num_features_per_split = int(np.ceil(p/3))

    rf_model_list = []

    # train one model for each component
    for j in range(d):
        c_vec = c[j, :]
        rf_model = RandomForestRegressor(n_estimators=num_trees, max_features=num_features_per_split)
        rf_model.fit(X_t, c_vec)
        rf_model_list.append(rf_model)

    return rf_model_list
import numpy as np

def predict_random_forests_po(rf_model_list, X_new):
    p, n = X_new.shape
    d = len(rf_model_list)
    X_new_t = X_new.T
    preds = np.zeros((d, n))
    for j in range(d):
        preds[j, :] = rf_model_list[j].predict(X_new_t)
    return preds


In [14]:
rf_model_list = train_random_forests_po(X_train, c_train, rf_graph(num_trees = 100, num_features_per_split = 2))
c_pred = predict_random_forests_po(rf_model_list, X_test)